Вас пригласили на работу в коммерческую компанию, занимающуюся разработкой автоматизированных торговых агентов. Одной из первых ваших задач будет подготовка данных для дальнейшей обработки и построения модели. Пообщавшись с коллегами, вы узнали, что вам предстоит работать с несколькими типами активов: акциями из списка SnP500 и криптовалютами (BTC, ETH, SOL, XRP). Вам планируют поручить краткосрочную и среднесрочную торговлю.


Вам предлагается на основе предоставленной информации:


1. Создать git-репозиторий, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.
2. Добавить файл лицензии, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.
3. Создать код на Python, который загрузит на ваш локальный компьютер данные о котировках ценных бумаг из списка SnP500 и котировки криптовалют (BTC, ETH, SOL, XRP).
4. Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте автоматическое отображение графиков текущей ситуации.
5. Проверьте нет ли в данных пропусков или ошибок. Проанализируйте выбросы. Оцените, на самом ли деле это выбросы или реальные данные, с которыми предстоит работать.

In [ ]:
# Initialization cell

%pip install yfinance -qq
%pip install pandas_datareader pandas numpy plotly seaborn matplotlib scipy -qq
from scipy.stats import zscore
from typing import Dict
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_datareader as web
import plotly.graph_objects as go
import seaborn as sns
import yfinance as yf


# Create Data directory if it doesn't exist
dir_stocks = os.path.join('data', 'stock')
if not os.path.exists(dir_stocks):
    os.makedirs(dir_stocks)
dir_crypto = os.path.join('data', 'crypto')
if not os.path.exists(dir_crypto):
    os.makedirs(dir_crypto)

In [ ]:
# Data download to subfolders cell

# Download SnP500 sub-tickers data
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.','-').to_list()[:10]
data: Dict[str, pd.DataFrame] = yf.download(tickers=tickers, group_by='Ticker', multi_level_index=False,progress=False)
data_per_ticker = {}
for ticker in tickers:
    df = data[ticker]#.dropna()
    df.columns = [x.lower() for x in df.columns]
    df.index.name = df.index.name.lower()
    df.to_csv(os.path.join(dir_stocks, ticker + '.csv'))


# Download crypto data
cryptos = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD']
for crypto in cryptos:
    data = yf.download(crypto, multi_level_index=False, progress=False)#.dropna()
    df.columns = [x.lower() for x in df.columns]
    df.index.name = df.index.name.lower()
    data.to_csv(os.path.join(dir_crypto, crypto.split("-")[0] + '.csv'))


In [ ]:
# Functions definition for load local and plot data cell

def load_data(ticker: str, is_crypto: bool = False):
    df: pd.DataFrame = None
    path = dir_stocks
    if is_crypto:
        path = dir_crypto
    path = os.path.join(path, ticker + '.csv')
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=['date'])
    else:
        print(f'No file for {ticker} at {path}')
    return df

def plot(df, time_start = None, time_end = None):
    if time_start is None:
        time_start = min(df['date'])
    if time_end is None:
        time_end = max(df['date'])
    df = df[(df['date'] >= time_start) & (df['date'] <= time_end)]
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
    fig.show()


In [ ]:
# Functions definition for data cleaning cell

def clean_data(ticker: str, df:  pd.DataFrame):
    df = drop_nones(ticker, df)
    df = drop_high_dispersion(ticker, df)
    df = drop_price_anomalies(ticker, df)
    return df

def drop_nones(ticker: str, df:  pd.DataFrame):
    # Удаляем пропуски (None)
    initial_length = len(df)
    df = df.dropna()
    if len(df) != initial_length:
        print(f' Удалено {initial_length - len(df)} пустых значений для {ticker}')
    return df.reset_index(drop=True)

def drop_high_dispersion(ticker: str, df: pd.DataFrame, threshold: float = 0.1) -> None:
    # Удаляем данные с высокой дисперсией данных, (High - Low) / ((High + Low) / 2) > threshold
    dispersion = (df['high'] - df['low']) / ((df['high'] + df['low']) / 2)
    # Создаём фильтр
    valid_mask = dispersion <= threshold
    # Применяем фильтр
    df = df[valid_mask]
    removed_count = (~valid_mask).sum()
    if removed_count > 0:
        print(f" Удалено {removed_count} записей с высокой дисперсией для {ticker}")
    return df

def drop_price_anomalies(ticker: str, df: pd.DataFrame, 
                            z_threshold: float = 3.0, 
                            window: int = 20) -> None:
    # Удаляем данные, применяя z-score для МА(window) (если данные более чем на 3 стандартных отклонения - в мусор)
    price_columns = ['open', 'high', 'low', 'close']
    # Создаём фильтр
    valid_mask = pd.Series(True, index=df.index)
    for col in price_columns:
        # Рассчитываем МА(window)
        rolling_mean = df[col].rolling(window=window, center=True).mean()
        rolling_std = df[col].rolling(window=window, center=True).std()
        # Рассчет отклонения
        z_scores = np.abs((df[col] - rolling_mean) / rolling_std)
        # Убираем из фильтра неподходящие значения
        valid_mask &= (z_scores <= z_threshold)
    # Применяем фильтр
    df = df[valid_mask]
    removed_count = (~valid_mask).sum()
    if removed_count > 0:
        print(f" Удалено {removed_count} аномальных цен {ticker}")
    return df


In [ ]:
# Example of load local data and plot
ticker = 'A'
df = load_data(ticker)
time_start = None#pd.to_datetime('2000-01-01')
time_end   = None#pd.to_datetime('2000-01-05')
plot(df, time_start, time_end)

In [ ]:
# Example of load local data and clean it up
ticker = 'A'
df = load_data(ticker)
df = clean_data(ticker, df)
time_start = None#pd.to_datetime('2000-01-01')
time_end   = None#pd.to_datetime('2000-01-05')
plot(df, time_start, time_end)